'''[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)]
(https://studiolab.sagemaker.aws/import/github/schumadi/nd025-disaster-response/blob/master/notebooks/ML Pipeline Preparation.ipynb)'''

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

import pandas as pd
import numpy as np
import re
import pickle

from scipy import stats

import optuna
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, CategoricalDistribution

from sqlalchemy import create_engine

from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk;nltk.download('popular');nltk.download('stopwords')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, Dataset

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


from sklearn.metrics import precision_recall_fscore_support

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/disaster.db')
df = pd.read_sql_table('Messages', engine)

X = df['message']
y = df.drop(columns=['message'])

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """_summary_

    Args:
        text (_type_): _description_

    Returns:
        _type_: _description_
    """
    # prep nltk transformation objects
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "URL")

    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmed = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]


    return lemmed 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer = tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth=7, min_samples_split=5)))
                    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fb4aaad4b80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier(max_depth=7,
                                                                        min_samples_split=5,
                                                                        random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_hat = pipeline.predict(X_test)

In [8]:
def report_f1_score_precision_recall(y_test, y_hat):
        row_list = []
        for col_number, col_name in enumerate(y_test.columns):
                for label in np.unique(y_hat.ravel()):

                        # Get the precision, reacall, f_score and support for column number col_number
                        pre_rec_f1_supp = precision_recall_fscore_support(y_true = y_test.iloc[:,col_number], y_pred = y_hat[:,col_number])
                        # The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label a negative sample as positive.
                        # The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
                        # The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.
                        # The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.
                        # The support is the number of occurrences of each class in y_true.

                        # Collect the results. The dicts are collected in row_list which is converted to a Dataframe at the end.

                        result = { 'Label': col_name, 'Value': label, 'Precision': pre_rec_f1_supp[0][label], 'Recall': pre_rec_f1_supp[1][label],
                                'F-1 Score': pre_rec_f1_supp[2][label], 'Support': pre_rec_f1_supp[3][label] }
                        row_list.append(result)
        for a in ['micro', 'macro', 'weighted']:
                average = precision_recall_fscore_support(y_true = y_test, y_pred = y_hat, average=a)
                result = { 'Label': 'Average', 'Value': a, 'Precision': average[0], 'Recall': average[1],
                                'F-1 Score': average[2], 'Support': '-' }
                row_list.append(result)
        res_df=pd.DataFrame(row_list)
        res_df.set_index(['Label', 'Value'], inplace=True)
        return res_df

report_f1_score_precision_recall(y_test, y_hat)

Precision    Recall  F-1 Score Support
Label         Value                                           
related       0          0.479167  0.019008   0.036566    1210
              1          0.769961  0.993747   0.867657    3998
request       0          0.835441  0.979389   0.901706    4318
              1          0.390411  0.064045   0.110039     890
offer         0          0.995004  0.999228   0.997111    5182
...                           ...       ...        ...     ...
direct_report 0          0.812113  0.994314   0.894025    4221
              1          0.400000  0.016211   0.031159     987
Average       micro      0.720674  0.271667   0.394589       -
              macro      0.158851  0.045662   0.053415       -
              weighted   0.422777  0.271667   0.260445       -

[73 rows x 4 columns]

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# define the grid of values to search
parameters = dict()
# clf__estimator__ prefix to address the model in the pipeline above

parameters['clf__estimator__max_depth'] = [7, 9, 11]
parameters['clf__estimator__min_samples_split'] = [5, 7, 9]


# define the evaluation procedure
cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)

# define the grid search procedure
grid_search = GridSearchCV(verbose=3, estimator=pipeline, param_grid=parameters, n_jobs=1, cv=cv, scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Fitting 6 folds for each of 9 candidates, totalling 54 fits
[CV 1/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=5;, score=0.173 total time=  14.3s
[CV 2/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=5;, score=0.174 total time=  14.4s
[CV 3/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=5;, score=0.178 total time=  14.0s
[CV 4/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=5;, score=0.164 total time=  13.8s
[CV 5/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=5;, score=0.172 total time=  14.8s
[CV 6/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=5;, score=0.182 total time=  13.3s
[CV 1/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=7;, score=0.173 total time=  13.3s
[CV 2/6] END clf__estimator__max_depth=7, clf__estimator__min_samples_split=7;, score=0.174 total time=  13.3s
[CV 3/6] END clf__estimator__max_depth=7, clf__estim

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_hat2 = grid_result.best_estimator_.predict(X_test)

In [11]:
report_f1_score_precision_recall(y_test, y_hat2)

Precision    Recall  F-1 Score Support
Label         Value                                           
related       0          0.479167  0.019008   0.036566    1210
              1          0.769961  0.993747   0.867657    3998
request       0          0.835441  0.979389   0.901706    4318
              1          0.390411  0.064045   0.110039     890
offer         0          0.995004  0.999228   0.997111    5182
...                           ...       ...        ...     ...
direct_report 0          0.812113  0.994314   0.894025    4221
              1          0.400000  0.016211   0.031159     987
Average       micro      0.720674  0.271667   0.394589       -
              macro      0.158851  0.045662   0.053415       -
              weighted   0.422777  0.271667   0.260445       -

[73 rows x 4 columns]

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [5]:

# Use LGBM Classifier

pipeline2 = Pipeline([
                        ('vect', CountVectorizer(tokenizer = tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(LGBMClassifier(random_state=42, objective='binary', n_jobs=-1)))
                    ])


optuna_param_dist = {
                        'clf__estimator__num_leaves': IntUniformDistribution(20,40),
                        'clf__estimator__max_depth': IntUniformDistribution(3,15),
                        'clf__estimator__min_data_in_leaf': IntUniformDistribution(10,100)
                    }
                         
clf = OptunaSearchCV(pipeline2, optuna_param_dist, cv=5, random_state=42, verbose=2, n_jobs=-1)

clf.fit(X_train, y_train)

print("Best trial:")
trial = clf.study_.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-08-15 13:20:23,669] A new study created in memory with name: no-name-21ffda48-0e20-42fa-bfe0-49c8b3718641
[I 2022-08-15 13:20:23,670] Searching the best hyperparameters using 20830 samples...


[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] min_data_in_leaf is set=91, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=91
[LightGBM] [Warning] min_data_in_leaf is set=97, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=97
[LightGBM] [Warning] min_data_in_leaf is set=48, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=48
[LightGBM] [Warning] min_data_in_leaf is set=99, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=99
[LightGBM] [Warning] min

[I 2022-08-15 13:24:26,006] Trial 1 finished with value: 0.18206913106096975 and parameters: {'clf__estimator__num_leaves': 23, 'clf__estimator__max_depth': 4, 'clf__estimator__min_data_in_leaf': 97}. Best is trial 1 with value: 0.18206913106096975.


[LightGBM] [Warning] min_data_in_leaf is set=99, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=99
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82


[I 2022-08-15 13:25:31,762] Trial 0 finished with value: 0.17762842054728756 and parameters: {'clf__estimator__num_leaves': 21, 'clf__estimator__max_depth': 5, 'clf__estimator__min_data_in_leaf': 48}. Best is trial 1 with value: 0.18206913106096975.


...

[LightGBM] [Warning] min_data_in_leaf is set=68, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=68
[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] min_data_in_leaf is set=68, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=68


[I 2022-08-15 13:34:46,392] Trial 7 finished with value: 0.1642822851656265 and parameters: {'clf__estimator__num_leaves': 37, 'clf__estimator__max_depth': 10, 'clf__estimator__min_data_in_leaf': 29}. Best is trial 1 with value: 0.18206913106096975.
[I 2022-08-15 13:35:14,469] Trial 8 finished with value: 0.16577052328372538 and parameters: {'clf__estimator__num_leaves': 39, 'clf__estimator__max_depth': 9, 'clf__estimator__min_data_in_leaf': 21}. Best is trial 1 with value: 0.18206913106096975.
[I 2022-08-15 13:35:18,990] Trial 9 finished with value: 0.15847335573691793 and parameters: {'clf__estimator__num_leaves': 30, 'clf__estimator__max_depth': 14, 'clf__estimator__min_data_in_leaf': 68}. Best is trial 1 with value: 0.18206913106096975.
[I 2022-08-15 13:35:18,991] Finished hyperparemeter search!
[I 2022-08-15 13:35:18,994] Refitting the estimator using 20830 samples...


[LightGBM] [Warning] min_data_in_leaf is set=97, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=97


[I 2022-08-15 13:35:42,192] Finished refitting! (elapsed time: 23.197 sec.)


Best trial:
  Value:  0.18206913106096975
  Params: 
    clf__estimator__num_leaves: 23
    clf__estimator__max_depth: 4
    clf__estimator__min_data_in_leaf: 97


In [16]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [29]:
plot_parallel_coordinate(clf.study_)

In [24]:

y_hat = clf.predict(X_test)

report_f1_score_precision_recall(y_test, y_hat)

Precision    Recall  F-1 Score Support
Label         Value                                           
related       0          0.391304  0.007438   0.014599    1210
              1          0.768370  0.996498   0.867690    3998
request       0          0.833787  0.992126   0.906091    4318
              1          0.514286  0.040449   0.075000     890
offer         0          0.995008  1.000000   0.997498    5182
...                           ...       ...        ...     ...
direct_report 0          0.813737  0.993603   0.894720    4221
              1          0.500000  0.027356   0.051873     987
Average       micro      0.743292  0.274110   0.400518       -
              macro      0.149056  0.043701   0.050509       -
              weighted   0.407979  0.274110   0.264341       -

[73 rows x 4 columns]

### 9. Export your model as a pickle file

In [32]:

with open('../models/classifier.pkl', 'wb') as file:
    pickle.dump(clf, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.